# RDD

* [Conceitos iniciais](#inicio)
* [Leitura e Visualização](#leitura)
* [Transformações de Map e FlatMap](#map_flatmap)
* [Transformações de Filter e Reduce](#filter_reduce)
* [Transformações de Ordenação](#ordenacao)
* [Armazenameto e Visualização de Dados](#armazenamento)
* [Partições](#particoes)

## <a name="inicio"> Conceitos iniciais </a>

[voltar ao ínicio](#inicio)

In [1]:
!python -V

Python 3.6.10 :: Anaconda, Inc.


In [2]:
spark

In [3]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [4]:
spark.sparkContext.setLogLevel("INFO")

In [5]:
spark.catalog.listDatabases()

[Database(name='default', description='Default Hive database', locationUri='hdfs://namenode:8020/user/hive/warehouse')]

## <a name="leitura"> Leitura e visualização de dados </a>

[voltar ao ínicio](#inicio)

In [6]:
rdd = sc.textFile("/user/tatiana/data/entrada*")

In [7]:
rdd.count()

12

In [8]:
rdd.first()

'Big Data'

In [9]:
rdd.take(5)

['Big Data', '2019', 'Semantix', 'Hadoop', 'Semantix SP']

In [10]:
rdd.collect()

['Big Data',
 '2019',
 'Semantix',
 'Hadoop',
 'Semantix SP',
 'Hadoop 2019',
 'Curso Hadoop',
 'Data',
 'Hadoop',
 'Semantix',
 'SP',
 'Data']

In [11]:
rdd.take(10)

['Big Data',
 '2019',
 'Semantix',
 'Hadoop',
 'Semantix SP',
 'Hadoop 2019',
 'Curso Hadoop',
 'Data',
 'Hadoop',
 'Semantix']

## <a name="map_flatmap"> Transformações de Map e FlatMap </a>

[voltar ao ínicio](#inicio)

### FlatMap

In [12]:
rdd.take(2)

['Big Data', '2019']

In [13]:
palavras = rdd.flatMap(lambda x: x.split(" "))

In [14]:
palavras.collect()

['Big',
 'Data',
 '2019',
 'Semantix',
 'Hadoop',
 'Semantix',
 'SP',
 'Hadoop',
 '2019',
 'Curso',
 'Hadoop',
 'Data',
 'Hadoop',
 'Semantix',
 'SP',
 'Data']

### Função Anonima

In [15]:
rdd.take(2)

['Big Data', '2019']

In [16]:
p1 = rdd.flatMap(lambda x: x.split(" "))

In [17]:
def Func(linha):
    linha = linha.lower()
    return linha

In [18]:
minuscula = p1.map(Func)

In [19]:
minuscula.collect()

['big',
 'data',
 '2019',
 'semantix',
 'hadoop',
 'semantix',
 'sp',
 'hadoop',
 '2019',
 'curso',
 'hadoop',
 'data',
 'hadoop',
 'semantix',
 'sp',
 'data']

### Diferenças de Map e FlatMap

In [20]:
rdd.take(2)

['Big Data', '2019']

In [21]:
palavras = rdd.flatMap(lambda x: x.split(" "))

In [22]:
palavras.collect()

['Big',
 'Data',
 '2019',
 'Semantix',
 'Hadoop',
 'Semantix',
 'SP',
 'Hadoop',
 '2019',
 'Curso',
 'Hadoop',
 'Data',
 'Hadoop',
 'Semantix',
 'SP',
 'Data']

In [23]:
palavras = rdd.map(lambda x: x.split(" "))

### Transformações no Map

In [24]:
palavras1 = rdd.flatMap(lambda x: x.split(" "))

In [25]:
minuscula = palavras1.map(lambda linha: linha.lower())

In [26]:
maiuscula = palavras1.map(lambda linha: linha.upper())

In [27]:
pChaveValor = minuscula.map(lambda palavra: (palavra,1))

In [28]:
pChaveValor.take(4)

[('big', 1), ('data', 1), ('2019', 1), ('semantix', 1)]

## <a name="filter_reduce"> Transformações de Filter e Reduce </a>

[voltar ao ínicio](#inicio)

### Transformações de Filter

In [29]:
#filtrar palavras que começam com "a"
filtro_a = palavras1.filter(lambda palavra: palavra.startswith("a"))


In [30]:
#filtrar palavras por tamanho da palavra, é muito comum palavras acima de 2, para não pegar palavras como ou, e, a, virgulas, etc.
filtro_tamanho = palavras1.filter(lambda palavra: len(palavra)>5)


In [31]:
#no nosso caso não estamos trabalhando com número, mas seria um exemplo para utilização do lambda
#num_par = numero.filter(lambda numero: numero %2==0)


### Transformações de Reduce

In [32]:
p_chave_valor = minuscula.map(lambda palavra: (palavra, 1))

In [33]:
#reduzir a chave valor através da soma da quantidade de palavras  = mapReduce reduzindo várias palavras em apenas uma chave valor
# processo de word count
# separação de um texto em linhas
# linhas separadas por espaço = palavras
# juntar as palavras 
# somar as palavras reduzindo em apenas uma chave valor

p_reduce = p_chave_valor.reduceByKey(lambda key1, key2: key1 + key2)

In [34]:
p_reduce.take(3)

[('big', 1), ('semantix', 3), ('data', 3)]

## <a name="ordenacao"> Transformações de Ordenação </a>

[voltar ao ínicio](#inicio)

In [35]:
p_ordena = p_reduce.sortBy(lambda palavra: palavra[1], False)

In [36]:
p_ordena.take(4)

[('hadoop', 4), ('semantix', 3), ('data', 3), ('sp', 2)]

## <a name="armazenamento"> Armazenamento e Visualização </a>

[voltar ao ínicio](#inicio)

In [37]:
# o collect(lista) em Python, já traz de forma estrutrada o resultado

lista = p_ordena.collect()
for row in lista:
    print(row[0],"-", row[1])

hadoop - 4
semantix - 3
data - 3
sp - 2
2019 - 2
big - 1
curso - 1


In [38]:
p_ordena.getNumPartitions

<bound method PipelinedRDD.getNumPartitions of PythonRDD[33] at collect at <ipython-input-37-425a2df54e27>:3>

In [39]:
p_ordena.saveAsTextFile("saida")

Py4JJavaError: An error occurred while calling o310.saveAsTextFile.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory hdfs://namenode:8020/user/root/saida already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:131)
	at org.apache.spark.internal.io.HadoopMapRedWriteConfigUtil.assertConf(SparkHadoopWriter.scala:287)
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:71)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply$mcV$sp(PairRDDFunctions.scala:1096)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withS